In [1]:
import numpy as np
import pandas as pd
import torch


In [369]:
import math

In [591]:
df = pd.read_csv("del_1.csv")
#df = df[(df['i']% 20 == 0) & (df['i'] > 0)]

In [592]:
eps = 0.06
adivdiv = torch.tensor(-1,dtype=torch.float64, requires_grad = True)
adeldel = torch.tensor(-1,dtype=torch.float64,requires_grad = True)
adivdel = torch.tensor(1,dtype=torch.float64,requires_grad = True)
adeldiv = torch.tensor(1,dtype=torch.float64,requires_grad = True)
mudiv = torch.tensor(1,dtype=torch.float64,requires_grad = True)
mudel = torch.tensor(1,dtype=torch.float64,requires_grad = True)
bdivdiv = torch.tensor(1,dtype=torch.float64,requires_grad = True)
bdeldel = torch.tensor(1,dtype=torch.float64,requires_grad = True)
bdivdel = torch.tensor(1,dtype=torch.float64,requires_grad = True)
bdeldiv = torch.tensor(1,dtype=torch.float64,requires_grad = True)

In [593]:
for i in range(0,301,20):
    df[f"div{i}"] = 0
    df[f"del{i}"] = 0

In [594]:
for cell in df.index:
    i = df.at[cell,'i']
    if i % 20 != 0:
        continue
    cell_id = df.at[cell,'id']
    for j in range(0,i,20):
        snapshot = df[(df['i']==j)&(df['next']!="none")&(df['id']!=df.at[cell,'id'])]
        distances = np.sqrt(np.power(snapshot['x'] - df.at[cell,'x'],2) + np.power(snapshot['y'] - df.at[cell, 'y'],2))
        close = distances[distances < eps].index
        counts = snapshot[snapshot.index.isin(close)]['next'].value_counts()
        try:
            df.loc[cell, f"div{i - j}"] = counts['div']
        except:
            df.loc[cell, f"div{i - j}"] = 0
        try:
            df.loc[cell, f"del{i - j}"] = counts['del']
        except:
            df.loc[cell, f"del{i - j}"] = 0


In [ ]:
def kernel(dt, b):
    """
    Need to test this to be fair
    """
    bdt = torch.mul(b, dt)
    first = torch.pow(torch.mul(bdt, torch.sqrt(torch.mul(2, np.pi))),-1)
    num = - torch.pow(torch.log(0.5 * dt),2)
    den = torch.mul(2, torch.pow(b,2))
    second = torch.exp(torch.div(num, den))
    return torch.mul(first, second)

In [ ]:
def conditional_intensity_div(dels,mudiv, adivdiv, adeldiv, bdivdiv, bdeldiv):
    """
    pass divs, dels as lists/tensors 
    """
    intensity = torch.tensor(0,dtype=torch.float,requires_grad=True)
    #kernel_multipliers_div = torch.tensor([kernel(torch.tensor(i+1), bdivdiv) for i in range(len(divs))],dtype=torch.float64,requires_grad=True)
    #multipliers_div = torch.mul(kernel_multipliers_div, adivdiv)
    kernel_multipliers_del = torch.tensor([kernel(torch.tensor(i+1), bdeldiv) for i in range(len(dels))],dtype=torch.float64,requires_grad=True)
    multipliers_del = torch.mul(kernel_multipliers_del, adeldiv)
    #intensity = torch.add(intensity,torch.dot(multipliers_div, divs))
    intensity = torch.add(intensity, torch.dot(multipliers_del, dels))
    intensity = torch.add(intensity, mudiv)
    return intensity

In [569]:
def conditional_intensity_del(divs,mudel, adivdiv, adivdel, bdivdiv, bdivdel):
    """
    pass divs, dels as lists/tensors 
    """
    intensity = torch.tensor(0,dtype=torch.float,requires_grad=True)
    kernel_multipliers_div = torch.tensor([kernel(torch.tensor(i+1), bdivdel) for i in range(len(dels))],dtype=torch.float64,requires_grad=True)
    multipliers_div = torch.mul(kernel_multipliers_div, adivdel)
    intensity = torch.add(intensity, torch.dot(multipliers_div, divs))
    intensity = torch.add(intensity, mudel)
    return intensity

In [ ]:
def get_likelihood_div(cellid,mudiv, adivdiv, adeldiv, bdivdiv, bdeldiv):
    likelihood = torch.tensor(0, dtype=torch.float, requires_grad=True)
    cell = df[df['id']==cellid].copy()
    cell['intensity'] = 0
    for index in cell.index:
        #divs = torch.tensor([df.at[index,f"div{i}"] for i in range(20,301,20)],dtype=torch.float64)    
        dels = torch.tensor([df.at[index,f"del{i}"] for i in range(20,301,20)],dtype=torch.float64)
        cell.loc[index, 'intensity'] = conditional_intensity_div(dels,mudiv, adivdiv, adeldiv, bdivdiv, bdeldiv).item()
        if cell.at[index, 'next'] == "div":
            likelihood = torch.add(likelihood, torch.log(torch.tensor(cell.at[index,'intensity'])))
        likelihood = torch.add(likelihood,torch.mul(-1,cell.at[index,'intensity']))
    likelihood = torch.div(likelihood, len(cell))
    return likelihood

In [562]:
def get_likelihood_del(cellid,mudel, adivdiv, adeldiv, bdivdiv, bdeldiv):
    likelihood = torch.tensor(0, dtype=torch.float, requires_grad=True)
    cell = df[df['id']==cellid].copy()
    cell['intensity'] = 0
    for index in cell.index:
        divs = torch.tensor([df.at[index,f"div{i}"] for i in range(20,301,20)],dtype=torch.float64)    
        #dels = torch.tensor([df.at[index,f"del{i}"] for i in range(20,301,20)],dtype=torch.float64)
        cell.loc[index, 'intensity'] = conditional_intensity_del(divs,mudel, adivdiv, adivdel, bdivdiv, bdivdel).item()
        if cell.at[index, 'next'] == "del":
            likelihood = torch.add(likelihood, torch.log(torch.tensor(cell.at[index,'intensity'])))
        likelihood = torch.add(likelihood,torch.mul(-1,cell.at[index,'intensity']))
    likelihood = torch.div(likelihood, len(cell))
    return likelihood

In [546]:
def get_total_likelihood_del(x):
    mudel = torch.tensor(x[0],dtype=torch.float64)
    adivdiv = torch.tensor(0,dtype=torch.float64)
    adivdel = torch.tensor(x[1],dtype=torch.float64)
    bdivdiv = torch.tensor(0,dtype=torch.float64)
    bdivdel = torch.tensor(x[2],dtype=torch.float64)
    likelihood = torch.tensor(0, dtype=torch.float64, requires_grad=True)
    for c in ids:
        likelihood = torch.add(likelihood, get_likelihood_del(c, mudel, adivdiv, adivdel, bdivdiv, bdivdel))
    print (x, likelihood.item())
    return - likelihood.item()

In [ ]:
def get_total_likelihood(x):
    mudiv = torch.tensor(x[0],dtype=torch.float64)
    adivdiv = torch.tensor(0,dtype=torch.float64)
    adeldiv = torch.tensor(x[1],dtype=torch.float64)
    bdivdiv = torch.tensor(0,dtype=torch.float64)
    bdeldiv = torch.tensor(x[2],dtype=torch.float64)
    likelihood = torch.tensor(0, dtype=torch.float64, requires_grad=True)
    for c in ids:
        likelihood = torch.add(likelihood, get_likelihood_div(c, mudiv, adivdiv, adeldiv, bdivdiv, bdeldiv))
    print (x, likelihood.item())
    return - likelihood.item()

In [ ]:
from scipy.optimize import minimize

In [570]:
get_total_likelihood_del([0.1, 0.05, 0.1])

[0.1, 0.05, 0.1] -416.2093827266364


416.2093827266364

In [572]:
minimize(get_total_likelihood_del, [0.1, 0.05, 0.1], options={"maxiter":10},method="L-BFGS-B",bounds =[(0.00001,None), (0,None),(None,None)])

[0.1  0.05 0.1 ] -416.2093827266364
[0.10000001 0.05       0.1       ] -416.209384288249
[0.1        0.05000001 0.1       ] -416.2093827266364
[0.1        0.05       0.10000001] -416.2093827266364
[1.e-05 5.e-02 1.e-01] -638.5947619613273
[1.001e-05 5.000e-02 1.000e-01] -638.564978719749
[1.000000e-05 5.000001e-02 1.000000e-01] -638.5947619613273
[1.0000000e-05 5.0000000e-02 1.0000001e-01] -638.5947619613273
[0.06505534 0.05       0.1       ] -409.7934606508315
[0.06505535 0.05       0.1       ] -409.7934627829446
[0.06505534 0.05000001 0.1       ] -409.7934606508315
[0.06505534 0.05       0.10000001] -409.7934606508315
[0.0217439 0.05      0.1      ] -419.9032294183348
[0.02174391 0.05       0.1       ] -419.9032208224544
[0.0217439  0.05000001 0.1       ] -419.9032294183348
[0.0217439  0.05       0.10000001] -419.9032294183348
[0.05327689 0.05       0.1       ] -409.4438217171106
[0.0532769 0.05      0.1      ] -409.4438210236032
[0.05327689 0.05000001 0.1       ] -409.4438217171106


      fun: 409.43123286892927
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([3.46930733, 0.        , 0.        ])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 84
      nit: 3
     njev: 21
   status: 0
  success: True
        x: array([0.05820013, 0.05      , 0.1       ])

In [573]:
minimize(get_total_likelihood_del, [0.09886096, 0.04943048, 1.09999919], options={"maxiter":10},method="L-BFGS-B",bounds =[(0.00001,None), (0,None),(None,None)])

[0.09886096 0.04943048 1.09999919] -415.9132777729534
[0.09886097 0.04943048 1.09999919] -415.91327943275735
[0.09886096 0.04943049 1.09999919] -415.9132777729534
[0.09886096 0.04943048 1.0999992 ] -415.9132777729534
[1.00000000e-05 4.94304800e-02 1.09999919e+00] -638.5947619613273
[1.00100000e-05 4.94304800e-02 1.09999919e+00] -638.564978719749
[1.00000000e-05 4.94304900e-02 1.09999919e+00] -638.5947619613273
[1.0000000e-05 4.9430480e-02 1.0999992e+00] -638.5947619613273
[0.06424016 0.04943048 1.09999919] -409.7297889244023
[0.06424017 0.04943048 1.09999919] -409.7297897299622
[0.06424016 0.04943049 1.09999919] -409.7297889244023
[0.06424016 0.04943048 1.0999992 ] -409.7297889244023
[0.03159244 0.04943048 1.09999919] -413.727447258335
[0.03159245 0.04943048 1.09999919] -413.72744329420703
[0.03159244 0.04943049 1.09999919] -413.727447258335
[0.03159244 0.04943048 1.0999992 ] -413.727447258335
[0.05686635 0.04943048 1.09999919] -409.4100911384769
[0.05686636 0.04943048 1.09999919] -409

[0.05506279 0.04943049 1.09999919] -409.40992690346604
[0.05506279 0.04943048 1.0999992 ] -409.40992690346604


      fun: 409.40992690346604
 hess_inv: <3x3 LbfgsInvHessProduct with dtype=float64>
      jac: array([36.91379788,  0.        ,  0.        ])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 152
      nit: 3
     njev: 38
   status: 0
  success: True
        x: array([0.05506279, 0.04943048, 1.09999919])

In [527]:
for index in df.index:
    dels = torch.tensor([df.at[index,f"del{i}"] for i in range(20,301,20)],dtype=torch.float64)
    df.loc[index, 'intensity'] = conditional_intensity_del(dels,torch.tensor(0.09886096,dtype=torch.float64), adivdiv, torch.tensor(0.04943048,dtype=torch.float64), bdivdiv, torch.tensor(1.09999919,dtype=torch.float64)).item()

In [528]:
df['intensity'].min()

0.09886096

In [529]:
df['y'] = 0
df.loc[df['next']=="del",'y'] = 1

In [437]:
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import roc_auc_score

In [530]:
fpr, tpr, threshold = roc_curve(df['y'], df['intensity'])
print(auc(fpr,tpr))

0.5942505630266607


In [595]:
for index in df.index:
    divs = torch.tensor([df.at[index,f"div{i}"] for i in range(20,301,20)],dtype=torch.float64)
    df.loc[index, 'intensity'] = conditional_intensity_del(divs,torch.tensor(0.05506279,dtype=torch.float64), adivdiv, torch.tensor(0.04943048,dtype=torch.float64), bdivdiv, torch.tensor(1.09999919,dtype=torch.float64)).item()

In [596]:
df['y'] = 0
df.loc[df['next']=="del",'y'] = 1

In [597]:
fpr, tpr, threshold = roc_curve(df['y'], df['intensity'])
print(auc(fpr,tpr))

0.5644249245127643
